# L2HMC using eager execution in tensorflow

### Imports

In [1]:
import os
import sys
import numpy as np
import tensorflow as tf

import tf_eager.dynamics_eager as _l2hmc
import tf_eager.gauge_dynamics_eager as l2hmc
from tf_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian

In [2]:
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice

In [3]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [4]:
tfe = tf.contrib.eager

In [5]:
def train_one_iter(dynamics, x, optimizer, 
                   loss_fn=l2hmc.compute_loss, global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn
    )
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob

### Construct GaugeLattice with $U(1)$ gauge group

In [6]:
time_size, space_size, dim, beta, batch_size = (8, 8, 2, 10, 5)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type='U1')
u1_samples = u1_lattice.get_links_samples(batch_size, link_type='U1')
u1_samples_flat = [i.flatten() for i in u1_samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples_flat), dtype=tf.float32)

In [7]:
u1_energy_fn = u1_lattice.get_energy_function()
u1_dynamics = l2hmc.GaugeDynamics(u1_lattice, 
                                  minus_loglikelihood_fn=u1_energy_fn, 
                                  batch_size=batch_size, n_steps=5, eps=0.1)

In [8]:
_momentum = tf.random_normal(tf.shape(u1_samples))
_potential = np.array(u1_dynamics.potential(u1_samples_tensor, batch_size))
_kinetic = u1_dynamics.kinetic(_momentum)
_hamiltonian = u1_dynamics.hamiltonian(u1_samples_tensor, _momentum)
_grad_potential = np.array(u1_dynamics.grad_potential(u1_samples_tensor))
grad_potential = tf.convert_to_tensor(_grad_potential, dtype=tf.float32)
print(_potential)
print(_kinetic.numpy())
print(_hamiltonian.numpy())
print(_grad_potential[0][:10])

[8.8475424  9.03759327 8.7656859  8.96741004 8.74220645]
[71.624146 60.374023 75.64526  79.23539  70.17695 ]
[80.47169 69.41161 84.41095 88.2028  78.91916]
[-0.8253179   0.27775407 -0.26843998  0.21609522  0.1919355  -0.78935415
  0.3210893  -0.835102    0.16938058 -0.1477234 ]


In [11]:
site = u1_lattice.get_random_site()
mu = np.random.randint(u1_lattice.dim)
nu = np.random.randint(u1_lattice.dim)
links = u1_lattice.links
plaq_op, _sum = u1_lattice.plaquette_operator(links, site, mu, nu)

In [14]:
shape = u1_lattice.site_idxs
l1 = tuple(list(np.mod(site, shape)) + [mu])
l2 = tuple(list(np.mod((site + u1_lattice.bases[mu]), shape)) + [nu])
l3 = tuple(list(np.mod((site + u1_lattice.bases[nu]), shape)) + [mu])
l4 = tuple(list(np.mod(site, shape)) + [nu])

In [17]:
links1 = np.exp(1j * links[l1])
links2 = np.exp(1j * links[l2])
links3 = np.exp(1j * links[l3])
links4 = np.exp(1j * links[l4])

In [23]:
def links_mtx(links, idx): 
    return tf.constant(
        [[np.cos(links[idx]), -np.sin(links[idx])],
         [np.sin(links[idx]), np.cos(links[idx])]],  dtype=tf.float32
    )

In [24]:
links_mtx1 = links_mtx(links, l1)
links_mtx2 = links_mtx(links, l2)
links_mtx3 = links_mtx(links, l3)
links_mtx4 = links_mtx(links, l4)

In [28]:
prod12 = tf.matmul(links_mtx1, links_mtx2)
prod34 = tf.matmul(tf.transpose(tf.conj(links_mtx3)),
                   tf.transpose(tf.conj(links_mtx4)))
prod1234 = tf.matmul(prod12, prod34)
prod1234

<tf.Tensor: id=40062, shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 1.]], dtype=float32)>

<tf.Tensor: id=40061, shape=(2, 2), dtype=float32, numpy=
array([[ 0.81110305,  0.58490324],
       [-0.58490324,  0.81110305]], dtype=float32)>

#### Run L2HMC for $U(1)$ gauge model

In [72]:
train_dir = '../U1_logs/run_6/'
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)
train_iters = 50
record_loss_every = 10
save_steps = 10

learning_rate = tf.train.exponential_decay(1e-3, global_step, 1000, 
                                           0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
summary_writer = tf.contrib.summary.create_file_writer(train_dir)
loss_fn = l2hmc.compute_loss
samples = u1_samples_tensor

In [75]:
u1_dynamics.eps.numpy()

<tf.Variable 'eps:0' shape=() dtype=float32, numpy=0.1>

In [77]:
# 7m13s for 64 links with shape (2, 2, 2, 2, 4), batch_size = 2
# 19m53s for 64 links with shape (2, 2, 2, 2, 4), batch_size = 5
total_actions = []
for i in range(1, train_iters + 1):
    loss, samples, accept_prob = train_one_iter(
        u1_dynamics, 
        samples, 
        optimizer,
        loss_fn = loss_fn,
        global_step=global_step
    )
    _total_actions = u1_lattice.total_action(samples, batch_size)
    total_actions.extend(_total_actions)
    
    print(_total_actions)
    print("Iteration {}, loss {:.4f}, x_accept {:.4f}, eps {:.4f}, avg_S {:.4f}".format(
        i, loss.numpy(), accept_prob.numpy().mean(), u1_dynamics.eps.numpy(), np.mean(_total_actions)
    ))
    print('\n')
    
    if i % record_loss_every == 0:
        with summary_writer.as_default():
            with tf.contrib.summary.always_record_summaries():
                tf.contrib.summary.scalar("Training loss", loss, 
                                          step=global_step)
    
    if i % save_steps == 0:
        saved_path = checkpointer.save(file_prefix=os.path.join(train_dir, 
                                                                "ckpt"))
        print(f"Saved checkpoint to: {saved_path}")

[8.787382543087007, 9.20905767381191, 9.192772043248016, 8.989190890143316, 8.68016199891766]
Iteration 1, loss 904.0479, x_accept 0.0000, eps 0.0980, avg_S 8.9717


[8.787382543087007, 9.20905767381191, 9.192772043248016, 8.989190890143316, 8.68016199891766]
Iteration 2, loss -13370.0830, x_accept 0.0000, eps 0.0973, avg_S 8.9717


[8.787382543087007, 9.20905767381191, 9.192772043248016, 8.989190890143316, 8.68016199891766]
Iteration 3, loss -6530.7075, x_accept 0.0000, eps 0.0965, avg_S 8.9717


[8.787382543087007, 9.20905767381191, 9.192772043248016, 8.989190890143316, 8.68016199891766]
Iteration 4, loss -10856.3574, x_accept 0.0000, eps 0.0957, avg_S 8.9717


[8.787382543087007, 9.20905767381191, 9.192772043248016, 8.989190890143316, 8.68016199891766]
Iteration 5, loss -13664.5410, x_accept 0.0000, eps 0.0949, avg_S 8.9717


[8.787382543087007, 9.20905767381191, 9.192772043248016, 8.989190890143316, 8.68016199891766]
Iteration 6, loss -15779.7217, x_accept 0.0002, eps 0.0941, avg_S

<tf.Tensor: id=3279333, shape=(), dtype=bool, numpy=True>

Saved checkpoint to: ../U1_logs/run_6/ckpt-1
[1.8681197795085607, 9.20905767381191, -3.0754725849255924, -2.0075448577602715, 5.438775953720322]
Iteration 11, loss -16668.3066, x_accept 0.8002, eps 0.0909, avg_S 2.2866


[-1.523618264434238, 9.20905767381191, -2.980269870293948, -0.8585512718806663, 2.8531240535279117]
Iteration 12, loss -14362.8486, x_accept 0.8019, eps 0.0904, avg_S 1.3399


[-1.8606484302630024, 5.883154237720494, -2.0803541851540412, 0.3140361029654741, 0.38035262624422717]
Iteration 13, loss -15356.8672, x_accept 1.0000, eps 0.0901, avg_S 0.5273


[-2.660862479669352, 3.565150518901646, -0.12456402419290205, -0.27006821955243754, -1.5449004704132678]
Iteration 14, loss -16888.3555, x_accept 1.0000, eps 0.0897, avg_S -0.2070


[-3.142997766534488, -1.4058924672814703, -0.6437085804063827, -0.6761342013875643, -1.5174639569595456]
Iteration 15, loss -15642.3779, x_accept 1.0000, eps 0.0895, avg_S -1.4772


[-2.3529570333970096, -2.661715400715669, -0.867583389704426

<tf.Tensor: id=3279333, shape=(), dtype=bool, numpy=True>

Saved checkpoint to: ../U1_logs/run_6/ckpt-2
[-0.25163980979899236, -0.4698568374539419, -1.6202226988195132, -1.7380979487982888, -1.617988980181205]
Iteration 21, loss -13340.0215, x_accept 1.0000, eps 0.0886, avg_S -1.1396


[-1.7537542032077915, -0.29858951612065177, -1.3898562958153582, -0.5958444187417626, -3.293625856439273]
Iteration 22, loss -13975.4248, x_accept 1.0000, eps 0.0886, avg_S -1.4663


[-2.9923299842824544, 0.8597504807015259, -0.5035185748711228, -0.7953720511868594, -0.8811628995463253]
Iteration 23, loss -14241.9951, x_accept 1.0000, eps 0.0886, avg_S -0.8625


[-2.5770919996624193, -0.16267499389747797, -1.0399942472577095, 0.21611385322952018, -0.2884341586225974]
Iteration 24, loss -16609.0938, x_accept 1.0000, eps 0.0886, avg_S -0.7704


[-1.2351860726096977, -0.10214334664245445, -1.70933996560052, -0.6224145263642338, -0.4383389310290413]
Iteration 25, loss -14465.7422, x_accept 1.0000, eps 0.0886, avg_S -0.8215


[-0.6848683964053635, -0.0212329028484724

KeyboardInterrupt: 

In [46]:
total_actions = np.array(total_actions).reshape(train_iters, batch_size)

In [47]:
np.mean(total_actions, axis=1)

array([13.27659808, 13.07365309, 13.07365309, 13.07365309, 13.07365309,
       13.07365309, 12.56995857, 12.56995857, 11.6948818 , 11.6948818 ,
       11.04025663, 10.1116315 ,  8.30393781,  6.02170212,  4.37457255,
        1.97069106,  0.77623208, -0.28165939, -1.24480359, -1.09183565,
       -0.99144835, -0.9755141 , -1.21855816, -0.88553424, -0.35959427,
       -0.6124662 , -0.31900744, -0.29786256, -0.41500096, -1.08966561,
       -1.11395323, -0.83870639, -0.75677254, -0.11470314, -0.26520809,
       -0.85103595, -1.325062  , -1.2029996 , -0.83623051, -1.10287612,
       -0.95760358, -0.57170075, -0.23748234, -0.99011732, -1.72738515,
       -1.44816752, -1.36281338, -1.10624829, -0.5324351 ,  0.32979044])

In [50]:
print(samples[0].numpy().reshape(u1_lattice.links.shape))

[[[[[0.6059793  2.174656   0.39121056 2.8758416 ]
    [3.8716912  0.56082726 1.6411109  1.596311  ]]

   [[0.4104136  0.428376   3.8064911  0.7468725 ]
    [1.0877397  1.220398   4.857299   0.93723595]]]


  [[[2.854507   0.50058836 0.09032671 0.7565119 ]
    [2.4704466  1.2997208  0.85661435 0.40303546]]

   [[1.8294978  0.46194318 1.125256   3.1045437 ]
    [4.346581   3.46119    1.291667   0.77315813]]]]



 [[[[1.9028021  0.756763   0.24777377 6.0015783 ]
    [0.3051336  0.7273674  1.0695355  0.44132656]]

   [[0.7852452  3.6366775  0.47717005 3.5577178 ]
    [6.1400743  1.7912223  4.0611787  1.991279  ]]]


  [[[1.437264   4.3467774  0.8976823  4.8767915 ]
    [1.3664039  3.1249125  4.8176794  4.905148  ]]

   [[0.28135774 2.4260843  1.3426757  1.7272958 ]
    [0.69482154 0.47839203 0.50907886 4.6306844 ]]]]]


In [78]:
saved_path = checkpointer.save(file_prefix=os.path.join(train_dir, 
                                                        "ckpt"))
print(f"Saved checkpoint to: {saved_path}")

Saved checkpoint to: ../U1_logs/run_6/ckpt-3


In [16]:
import sys
help(sys.stdout.flush)

Help on method flush in module ipykernel.iostream:

flush() method of ipykernel.iostream.OutStream instance
    trigger actual zmq send
    
    send will happen in the background thread



In [11]:
loss, samples, accept_prob

(<tf.Tensor: id=315487, shape=(), dtype=float32, numpy=435.77283>,
 <tf.Tensor: id=169879, shape=(2, 64), dtype=float32, numpy=
 array([[0.3880173 , 0.5899346 , 0.7016848 , 0.5990745 , 0.5219401 ,
         0.9196011 , 0.9525441 , 0.9359869 , 0.25037813, 0.61384606,
         0.8853723 , 0.7008663 , 0.46281803, 0.8451578 , 0.6307719 ,
         0.7388011 , 0.8698082 , 0.25262702, 0.7657299 , 0.7845003 ,
         0.41359487, 0.15271497, 0.9564786 , 0.9552279 , 0.8520697 ,
         0.5839676 , 0.8863903 , 0.80490005, 0.09511092, 0.21265566,
         0.5685413 , 0.18593708, 0.2397108 , 0.8676617 , 0.27230152,
         0.8837739 , 0.01008116, 0.4798919 , 0.7584469 , 0.63647527,
         0.46477962, 0.71595037, 0.91022605, 0.6463619 , 0.52518564,
         0.48763356, 0.21342352, 0.4212268 , 0.289053  , 0.23400478,
         0.7312603 , 0.48150775, 0.398928  , 0.8025431 , 0.9205895 ,
         0.27285406, 0.85092205, 0.81339276, 0.21941423, 0.44695216,
         0.4644379 , 0.5926884 , 0.5382971 ,

### Construct GaugeLattice with SU(3) gauge group

In [6]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
gauge_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = gauge_lattice.get_links_samples(batch_size, link_type=link_type)

In [8]:
gauge_energy_fn = gauge_lattice.get_energy_function()
gauge_dynamics = l2hmc.GaugeDynamics(gauge_lattice, 
                                     minus_loglikelihood_fn=gauge_energy_fn, 
                                     batch_size=3, n_steps=5, eps=0.1)

In [15]:
gauge_lattice.links.shape

(2, 4, 4, 4, 4, 3, 3)

In [9]:
potential_arr = gauge_dynamics.potential(links_samples, batch_size)

[i.numpy() for i in potential_arr]

[1268.4961, 1267.7504, 1267.6804]

In [11]:
_momentum = tf.random_normal(tf.shape(links_samples))

In [13]:
gauge_dynamics.kinetic(_momentum).numpy()

array([2295.3035, 2392.9976, 2327.1277], dtype=float32)

In [14]:
_x = links_samples
#_momentum = tf.random_normal(tf.shape(_x))
_hamiltonian = gauge_dynamics.hamiltonian(_x, _momentum)
_hamiltonian

<tf.Tensor: id=143019, shape=(3,), dtype=float32, numpy=array([3563.7996, 3660.748 , 3594.808 ], dtype=float32)>

### Construct IsingLattice

In [88]:
ising_batch_size = 10
ising_lattice = IsingLattice(3, 4)
ising_samples = [ising_lattice._randomize() for _ in range(ising_batch_size)]

In [89]:
ising_energy_fn = ising_lattice.get_energy_function()
ising_dynamics = l2hmc.LatticeDynamics(ising_lattice, 
                                       minus_loglikelihood_fn=ising_energy_fn,
                                       batch_size=ising_batch_size, 
                                       n_steps=10, eps=0.1)
#dynamics = l2hmc.LDynamics(latt.sites.shape, minus_loglikelihood_fn=energy_fn, n_steps=10, eps=0.1)

In [90]:
ising_dynamics.potential(samples, batch_size)

[0, 24, 0, 24, -16, -8, 0, -8, -32, 24]

In [91]:
_iposition = ising_samples
_imomentum = tf.random_normal(tf.shape(_iposition))
_ihamiltonian = dynamics.hamiltonian(_iposition, _imomentum)
_ihamiltonian

<tf.Tensor: id=458093, shape=(10,), dtype=float32, numpy=
array([22.397621, 50.170174, 55.35661 , 28.805382, 52.04349 , 32.253098,
       45.301197, 75.20062 , 23.465696, 17.285112], dtype=float32)>

In [92]:
_isample = _iposition[0].reshape(ising_lattice.num_sites)
#dynamics.grad_potential(np.array(_position).reshape(-1, lattice.num_sites))

In [107]:
grad_pot = dynamics.grad_potential(ising_samples)

In [97]:
grad_pot

In [109]:
ising_jacobian = jacobian(dynamics.potential, ising_samples)

IndexError: list index out of range

In [110]:
%debug

> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/eager/backprop.py(533)decorated()
    531       result = nest.pack_sequence_as(result, flat_result)
    532     finally:
--> 533       tape.pop_tape(this_tape)
    534     def vjp(dy=None):
    535       if dy is not None:

ipdb> ll
    508   def decorated(*args, **kwds):
    509     """Computes the value and gradient of the decorated function."""
    510     parameter_positions = _get_arg_spec(f, params, args)
    511     assert not kwds, "The gradient function can't take keyword arguments."
    512     this_tape = tape.push_new_tape(persistent=persistent)
    513     try:
    514       sources = []
    515       args = [
    516           ops.convert_to_tensor(args[i])
    517           if i in parameter_positions else args[i]
    518           for i in range(len(args))
    519       ]
    520       args = _ensure_unique_tensor_objects(parameter_positions, args)
    521       for i in param

ipdb> arr = args
ipdb> arr
[<tf.Tensor: id=589788, shape=(4, 4, 4), dtype=int64, numpy=
array([[[-1,  1,  1,  1],
        [ 1,  1, -1,  1],
        [ 1, -1, -1,  1],
        [-1,  1, -1, -1]],

       [[-1, -1,  1,  1],
        [-1,  1,  1,  1],
        [ 1, -1, -1,  1],
        [ 1, -1, -1, -1]],

       [[ 1,  1, -1, -1],
        [ 1, -1, -1,  1],
        [-1,  1, -1, -1],
        [ 1,  1,  1,  1]],

       [[-1,  1, -1, -1],
        [-1,  1,  1, -1],
        [ 1, -1,  1,  1],
        [ 1,  1,  1, -1]]])>]
ipdb> ll
    508   def decorated(*args, **kwds):
    509     """Computes the value and gradient of the decorated function."""
    510     parameter_positions = _get_arg_spec(f, params, args)
    511     assert not kwds, "The gradient function can't take keyword arguments."
    512     this_tape = tape.push_new_tape(persistent=persistent)
    513     try:
    514       sources = []
    515       args = [
    516           ops.convert_to_tensor(args[i])
    517           if i in para

ipdb> i
1
ipdb> parameter_positions
range(0, 2)
ipdb> sources
[<tf.Tensor: id=589788, shape=(4, 4, 4), dtype=int64, numpy=
array([[[-1,  1,  1,  1],
        [ 1,  1, -1,  1],
        [ 1, -1, -1,  1],
        [-1,  1, -1, -1]],

       [[-1, -1,  1,  1],
        [-1,  1,  1,  1],
        [ 1, -1, -1,  1],
        [ 1, -1, -1, -1]],

       [[ 1,  1, -1, -1],
        [ 1, -1, -1,  1],
        [-1,  1, -1, -1],
        [ 1,  1,  1,  1]],

       [[-1,  1, -1, -1],
        [-1,  1,  1, -1],
        [ 1, -1,  1,  1],
        [ 1,  1,  1, -1]]])>]
ipdb> arr[i]
*** IndexError: list index out of range
ipdb> len(arr)
1
ipdb> arr[0]
<tf.Tensor: id=589788, shape=(4, 4, 4), dtype=int64, numpy=
array([[[-1,  1,  1,  1],
        [ 1,  1, -1,  1],
        [ 1, -1, -1,  1],
        [-1,  1, -1, -1]],

       [[-1, -1,  1,  1],
        [-1,  1,  1,  1],
        [ 1, -1, -1,  1],
        [ 1, -1, -1, -1]],

       [[ 1,  1, -1, -1],
        [ 1, -1, -1,  1],
        [-1,  1, -1, -1],
        [ 1,  1,  

In [11]:
grad_fn = tfe.gradients_function(lattice._calc_energy, params=[0])

In [83]:
_jacobian = jacobian(dynamics.potential, _position)

IndexError: list index out of range

In [84]:
%debug

> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/eager/backprop.py(533)decorated()
    531       result = nest.pack_sequence_as(result, flat_result)
    532     finally:
--> 533       tape.pop_tape(this_tape)
    534     def vjp(dy=None):
    535       if dy is not None:

ipdb> ll
    508   def decorated(*args, **kwds):
    509     """Computes the value and gradient of the decorated function."""
    510     parameter_positions = _get_arg_spec(f, params, args)
    511     assert not kwds, "The gradient function can't take keyword arguments."
    512     this_tape = tape.push_new_tape(persistent=persistent)
    513     try:
    514       sources = []
    515       args = [
    516           ops.convert_to_tensor(args[i])
    517           if i in parameter_positions else args[i]
    518           for i in range(len(args))
    519       ]
    520       args = _ensure_unique_tensor_objects(parameter_positions, args)
    521       for i in param

ipdb> pp a
*** NameError: name 'a' is not defined
ipdb> pp _a
[<tf.Tensor: id=457719, shape=(200, 2), dtype=float64, numpy=
array([[-1.18493387e+00, -5.12083440e-02],
       [-5.07364717e-02,  9.94068867e-01],
       [-1.25786833e-01,  8.10973510e-01],
       [ 1.33987689e+00,  3.73350215e-02],
       [-2.77535839e-01, -7.18694900e-01],
       [-5.71033619e-02,  1.02224159e+00],
       [ 7.53654219e-01,  1.60942892e-02],
       [ 6.99635462e-01, -9.31503644e-02],
       [ 1.56398174e-01, -1.02645151e+00],
       [-5.78893863e-02, -7.55472792e-01],
       [ 6.20771547e-02, -8.91120846e-01],
       [ 1.05817367e+00,  3.34769684e-02],
       [-9.39682147e-01, -6.25299362e-03],
       [ 1.26956924e-02,  1.14243685e+00],
       [ 9.55679973e-01, -7.95782812e-02],
       [-2.93435906e-01,  9.65287675e-01],
       [ 9.13075338e-01,  3.59116436e-02],
       [ 7.80289232e-02,  9.93157338e-01],
       [-7.41820904e-02,  1.20525869e+00],
       [ 6.10620798e-02, -1.16368112e+00],
       [-4.33124

ipdb> _a.shape
*** AttributeError: 'list' object has no attribute 'shape'
ipdb> _a[0].shape
TensorShape([Dimension(200), Dimension(2)])
ipdb> _a
[<tf.Tensor: id=457719, shape=(200, 2), dtype=float64, numpy=
array([[-1.18493387e+00, -5.12083440e-02],
       [-5.07364717e-02,  9.94068867e-01],
       [-1.25786833e-01,  8.10973510e-01],
       [ 1.33987689e+00,  3.73350215e-02],
       [-2.77535839e-01, -7.18694900e-01],
       [-5.71033619e-02,  1.02224159e+00],
       [ 7.53654219e-01,  1.60942892e-02],
       [ 6.99635462e-01, -9.31503644e-02],
       [ 1.56398174e-01, -1.02645151e+00],
       [-5.78893863e-02, -7.55472792e-01],
       [ 6.20771547e-02, -8.91120846e-01],
       [ 1.05817367e+00,  3.34769684e-02],
       [-9.39682147e-01, -6.25299362e-03],
       [ 1.26956924e-02,  1.14243685e+00],
       [ 9.55679973e-01, -7.95782812e-02],
       [-2.93435906e-01,  9.65287675e-01],
       [ 9.13075338e-01,  3.59116436e-02],
       [ 7.80289232e-02,  9.93157338e-01],
       [-7.41820904

ipdb> _a[0].shape
TensorShape([Dimension(200), Dimension(2)])
ipdb> q


In [12]:
lattice.calc_energy(_position, batch_size)

[8, 40, -48, -40, 8, -32, 32, 16, -16, -8]

In [13]:
#dynamics.position_fn(momentum, latt.sites.flatten()[:], dynamics)
#dynamics._forward_lf(latt.sites.flatten()[:], momentum, 0)
dynamics._forward_lf(np.array(_position).reshape(-1, lattice.num_sites),
                     np.array(_momentum).reshape(-1, lattice.num_sites), 1)

InvalidArgumentError: Index out of range using input dim 1; input has only 1 dims [Op:StridedSlice] name: strided_slice/

### GMM Model

In [39]:
sigmas, distribution = gen_ring(1., var=0.02, nb_mixtures=4)

gmm_potential = distribution.get_energy_function()
gmm_dynamics = _l2hmc.Dynamics(x_dim=2, minus_loglikelihood_fn=gmm_potential,
                               n_steps=25, eps=0.1)

In [37]:
samples = distribution.get_samples(200)

In [45]:
_position = samples
_momentum = tf.random_normal(tf.shape(_position))
_hamiltonian = gmm_dynamics.hamiltonian(_position, _momentum)

In [49]:
grad_pot = gmm_dynamics.grad_potential(_position, _momentum)

In [51]:
grad_pot.shape

TensorShape([Dimension(200), Dimension(2)])